This notebook exists to investigate the need of the parameters included in the dataset. Each parameter is tested against the others to see if they are statistically correlated....
Then wrapper methods are used to determine which ones are most important....
Want to reduce number of parameters needed to produce a good output so to reduce the chance of overfitting and increase the generalisation of the model.

In [1]:
%run "Parameter_Estimation.ipynb" #allowing access to parameters

  0%|                                                                                          | 0/549 [00:00<?, ?it/s]

Filtering Database


 15%|████████████▎                                                                   | 34/221 [00:00<00:00, 334.62it/s]

221 remaining out of 290


100%|███████████████████████████████████████████████████████████████████████████████| 221/221 [00:00<00:00, 366.23it/s]


Normalising and preproccessing signals using DWT
Calculating Time Domain Features
Calculating Frequency Domain Features
Calculating Non-Linear Domain Features


In [2]:
from scipy.stats import pearsonr #for comparisson of correlations
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, mutual_info_classif #for forward feature selection
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [3]:
def get_balanced_accuracy(y_test, y_pred):
    """
    balanced accuracy....
    """
    num_healthy_true = np.sum([x=='Healthy' for x in y_test])
    num_unhealthy_true = len(y_test) - num_healthy_true
    count_healthy_accurate = 0
    count_unhealthy_accurate = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            count_unhealthy_accurate +=1
        elif y_pred[i] == y_test[i] == 'Healthy':
            count_healthy_accurate +=1
    healthy_percentage = count_healthy_accurate/num_healthy_true
    unhealthy_percentage = count_unhealthy_accurate/num_unhealthy_true
    balanced_accuracy = (healthy_percentage + unhealthy_percentage) * 0.5
    return balanced_accuracy

def get_specificity(y_test, y_pred):
    """
    true negative rate
    """
    true_negative = 0
    false_positive = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Healthy':
            true_negative += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
    
    return true_negative / (true_negative+false_positive)
            
def get_f1_score(y_test, y_pred):
    """
    balance between precision and recall
    """
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            true_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Unhealthy':
            false_negative += 1
    return (2*true_positive)/(2*true_positive+false_positive+false_negative)

def scoring_function(model, X, y):
    """
    change to incorporate balanced accuracy 
    """
    
    y_pred = model.predict(X)
    y_test = y
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    #specificity = get_specificity(y_test, y_pred)
    f1 = get_f1_score(y_test, y_pred)
    
    
    return f1*0.7 + balanced_acc*0.3

def convert_multi_dict_to_array(params, nan_indices):
    no_features = len(params)
    params_list = []
    for j in range(0, no_channels):
        params_array = np.zeros(shape=(len(health_state[nan_indices[j]]), no_features))
        for i, values in enumerate(params.values()):
            params_array[:, i] = values[j]
        params_list.append(params_array)
    return params_list, no_features

def convert_single_dict_to_array(params, nan_indices):
    no_features = len(params)
    params_array = np.zeros((len(health_state[nan_indices]), no_features))
    for i, values in enumerate(params.values()):
        params_array[:, i] = values
    return params_array, no_features

def reduce_parameter_set(params, selected_indices, channel):
    select_params = {}
    for i, key in enumerate(params.keys()):
        if i in selected_indices:
            select_params[key] = params[key][channel]
    return select_params
        
def reduce_parameter_set_single(params, selected_indices):
    select_params = {}
    for i, key in enumerate(params.keys()):
        if i in selected_indices:
            select_params[key] = params[key]
    return select_params
    

First convert the parameters to an array for easy investigation.

In [4]:
#pre-calculation of relevant health array for every channel
nan_indices = []
health_state = allowed_patients.get_diagnoses()
for j in range(0, no_channels):
    signal_nan_indices = []
    for i, signal in enumerate(denoised_signals[:, j]):
        if np.isnan(signal).all():
            signal_nan_indices.append(False)
        else:
            signal_nan_indices.append(True)
    nan_indices.append(signal_nan_indices)
    
health_state = np.array(health_state)
nan_indices = np.array(nan_indices)

In [5]:
params_array, no_features = convert_multi_dict_to_array(params, nan_indices)

desired_no_feats = 4

In [6]:
print(f'Selecting the {desired_no_feats} most important features from {no_features} for each channel')

Selecting the 4 most important features from 21 for each channel


To select the most important features to be used in analysis used forward sfs

In [8]:
def forward_selection(params, health_state, scorer=scoring_function, k=6):
    
    X_train, X_test, y_train, y_test = train_test_split(params, health_state, test_size=0.3, stratify = health_state)

    #initialise svc
    svm = SVC(class_weight='balanced')

    SFS_forward = SequentialFeatureSelector(estimator=svm, scoring=scoring_function, cv=3, n_features_to_select=k)

    SFS_forward.fit(X_train, y_train)

    selected_indices = SFS_forward.get_support(indices=True)
    
    return selected_indices

Filter methods

In [9]:
def filter_method(params, health_state, k=6, scorer=mutual_info_classif):
    # Apply SelectKBest with mutual information
    selector = SelectKBest(score_func=scorer, k=k)
    
    
    X_new = selector.fit_transform(params, health_state)
    
    # Get the indices of selected features
    selected_indices = selector.get_support(indices=True)
    
    return selected_indices

Combining approaches

In [10]:
selected_params = {}
for i in range(0, no_channels):
    #find indices to keep through filter method
    
    chosen_indices_filt = filter_method(params_array[i], health_state[nan_indices[i]], k=8)
    
    #reduce the parameter based on these indices
    reduced_params = reduce_parameter_set(params, chosen_indices_filt, i)

    #convert new params dict back into array
    reduced_params_array, no_features = convert_single_dict_to_array(reduced_params, nan_indices[i])

    #find 4 best features through sfs
    chosen_indices_filt_sfs = forward_selection(reduced_params_array, health_state[nan_indices[i]], scorer=scoring_function, k=desired_no_feats)

    selected_params[i] = reduce_parameter_set_single(reduced_params, chosen_indices_filt_sfs)
    

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
for i in range(0, no_channels):
    print(f"Selected features for channel {i}:")
    print(list(selected_params[i].keys()))